In [1]:
import arcpy
import pandas as pd
import numpy as np
import os

from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [3]:
# download each state's file to a folder (here called "states"). Set this folder as the initial workspace. There should be a geodatabase for each state/territory
arcpy.env.workspace = r"E:\EDF\wotus_wetlands\data\NWI\states"
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("USA Contiguous Albers Equal Area Conic USGS")
arcpy.env.parallelProcessingFactor = "100%"
scratch = r"E:\EDF\wotus_wetlands\scratch\scratch.gdb"

In [4]:
workspaces = arcpy.ListWorkspaces("*", "FileGDB")
outputGDB = r"E:\EDF\wotus_wetlands\data\NWI\combined\nwi_mosaic.gdb"
outputClippedGDB = r"E:\EDF\wotus_wetlands\data\NWI\combined\nwi_clipped_mosaic.gdb"
outputMergedGDB = r"E:\EDF\wotus_wetlands\analysis\Input\nwi_proj.gdb"

In [9]:
states = r"E:\EDF\wotus_wetlands\data\Census\tlgdb_2022_a_us_nationgeo.gdb\State"
states_proj = arcpy.conversion.ExportFeatures(states, os.path.join(scratch, "states_proj")) 

state_codes_include = ['AL','AZ','AR','CA','CO','CT','DC','DE','FL','GA', 'ID','IL','IN','IA','KS','KY','LA','MA','MD','ME','MI','MN','MS','MO','MT','NC','NE','NJ','NV','NM','ND','NH','NY','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY']
state_codes_include.sort()

In [11]:
for w in workspaces:
    
    # Extract state code
    state_code =  os.path.basename(w).split("_")[0]
    
    if state_code in state_codes_include:
        
        print(state_code)
        
        # Create path for regular wetlands feature class
        reg_wetlands = os.path.join(w, state_code + "_Wetlands")
        repaired_wetlands = os.path.join(outputGDB, state_code+"_Wetlands")
        clipped_wetlands = os.path.join(outputClippedGDB, state_code+"_Wetlands")
        merged_wetlands = os.path.join(outputMergedGDB, "merged_wetlands_conus")
        
        print("Multipart to singlepart and repair")
        # multipart to singlepart
        arcpy.management.MultipartToSinglepart(reg_wetlands, repaired_wetlands)
        
        # Repair geometry
        arcpy.management.RepairGeometry(repaired_wetlands)
        
        # Select the corresponding state geometry and clip wetlands layer to that extent
        print("Selecting state...")
        select_state = os.path.join(scratch, "select_state")
        
        arcpy.analysis.Select(states_proj, select_state, "STUSPS = '{}'".format(state_code))
        
        print("Clipping and calculating new areas...")
        arcpy.analysis.PairwiseClip(repaired_wetlands, select_state, clipped_wetlands)
        arcpy.management.CalculateGeometryAttributes(clipped_wetlands, [["ACRES","AREA"]], "KILOMETERS","ACRES_US")
        arcpy.management.CalculateField(clipped_wetlands, "STATE", "'{}'".format(state_code), "PYTHON3")
        
print("All done!")

AL
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
AR
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
AZ
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
CA
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
CO
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
IA
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
ID
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
IL
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
IN
Multipart to singlepart and repair
Selecting state...
Clipping and calculating new areas...
writing layer
KS
Multipart to sin